In [ ]:
import pandas as pd
from elasticsearch import Elasticsearch
from pandas import json_normalize
import plotly.express as px
from datetime import datetime

In [ ]:
es = Elasticsearch(hosts="http://127.0.0.1:9200")

In [ ]:
res = es.search(query={
    "bool": {
        "must": [
            { "exists": { "field": "party" }},
            { "match": { "active": "true" }},
        ],
    }},
    index="parliament.politicians", size=1000)
data = json_normalize(res["hits"]["hits"])

In [ ]:
others_party_list = list(data["_source.party"].value_counts()[data["_source.party"].value_counts() < 10].index)
data.loc[data["_source.party"].isin(others_party_list), "_source.party"] = "Sonstige"


In [ ]:
px.sunburst(data, path=["_source.party", "_source.gender"])

In [ ]:
data.head()

In [ ]:
data["_source.birthDate"] = pd.to_datetime(data["_source.birthDate"])
data["age"] = (datetime.now().year - data["_source.birthDate"].dt.year)

In [ ]:
px.histogram(data, x="age", color="_source.party", barmode="group", histnorm="percent")